# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
# import os
import chromadb
from typing import List
from tavily import TavilyClient
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [3]:
# TODO: Load environment variables
load_dotenv()

model="gpt-4o-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created

# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
@tool 
def retrieve_game(query: str):
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    results = collection.query(query_texts=[query], n_results=2)
    retrieved_docs = results['documents'][0]
    return {"documents": retrieved_docs}

#### Evaluate Retrieval Tool

In [5]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

class EvaluationOutput(BaseModel):
    is_data_sufficient: bool = Field(..., description="Boolean to indicate if the provided documents are enough for answering the question.")
    confidence_score: float = Field(..., description="Confidence Score with range from 0.0 to 1.0 reperesenting confidence in assessment.")

evaluation_instructions = """You are an expert AI game analyst. Your task is to determine if the provided DOCUMENTS string contains enough information to fully and accurately answer the user's QUESTION.

Your evaluation should be based on the following criteria:
* **Directness:** Does the document directly address the user's question?
* **Completeness:** Does the document provide a complete answer, or is it missing key details?
* **Timeliness:** For questions about recent events (like patches, releases, or news), is the information up-to-date? Information that is clearly old or superseded is not sufficient.

You MUST provide your response as a single, valid JSON object with the following two properties:

1.  `is_data_sufficient`: A boolean (`true` or `false`). Set to `true` only if the documents provide a complete and accurate answer.
2.  `confidence_score`: A float between 0.0 and 1.0 representing your confidence in your `is_data_sufficient` assessment.
"""
@tool 
def evaluate_retrieval(question: str, retrieved_docs: str):
    user_query=f"""
        User Question: {question},
        Retrieved Documents: {retrieved_docs}
    """
    messages = [
        SystemMessage(content=evaluation_instructions),
        UserMessage(content=user_query)
    ]
    evaluator = LLM(model=model, api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    results = evaluator.invoke(input=messages, response_format=EvaluationOutput)
    return results.content
    

#### Game Web Search Tool

In [6]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.
@tool 
def game_web_search(question: str):
    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    search_results = tavily.search(question)
    return search_results

### Agent

In [7]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
agent_instructions = """
You are a specialized AI assistant for video game information. Your primary goal is to provide the most accurate and comprehensive answer to a user's question about games. You must follow a strict, iterative process:

1.  **Initial Vector Search:** Given the user's query, your first action is ALWAYS to search the internal vector database using `retrieve_game` tool to find relevant information.

2.  **Critical Evaluation:** Once you have the results from the vector database, you MUST critically evaluate them using `evaluate_retrieval` tool to check if the created response is sufficient to answer user's question.

3.  **Web Search Augmentation (If Necessary):**
    * If you determine the information from the vector database is insufficient, you MUST use the `game_web_search` tool to find more current or comprehensive data.
    * Formulate a precise search query based on the gaps in the information you identified.

4.  **Synthesize and Answer:**
    * Combine the information from the initial vector search with the new data from your web search.
    * Once you are confident you have a complete and accurate answer, provide it to the user, clearly citing the sources of your information if possible.
"""

tools = [retrieve_game, evaluate_retrieval, game_web_search]
agent = Agent(model_name=model, api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL, instructions=agent_instructions, tools=tools)


In [8]:
def get_final_answer(messages: List[BaseMessage]):
    final_response = ""
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content:
            final_response = msg.content
    return final_response

def get_tools_called(messages: List[BaseMessage]):
    tools_called = []
    for msg in (messages):
        if isinstance(msg, ToolMessage) and msg.name:
            tools_called.append(msg.name)
    return tools_called

In [9]:
query1="When were Pokémon Gold and Silver released?"
sessionid1="pokemon"

run1 = agent.invoke(
    query=query1,
    session_id=sessionid1
)

messages1 = run1.get_final_state()["messages"]
print("="*150)
print(get_final_answer(messages1))
print("="*150)
print(get_tools_called(messages1))
print("="*150)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
**Pokémon Gold and Silver** were released on the following dates:

- **Japan**: November 21, 1999
- **North America**: October 15, 2000
- **Europe**: April 6, 2001

These games marked the second generation of Pokémon games and were released for the Game Boy Color. For more detailed information, you can visit the [Wikipedia page on Pokémon Gold and Silver](https://en.wikipedia.org/wiki/Pok%C3%A9mon_Gold_and_Silver).
['retrieve_game', 'evaluate_retrieval', 'game_web_sea

In [10]:
query2="Which one was the first 3D platformer Mario game?"
sessionid2="mario"

run2 = agent.invoke(
    query=query2,
    session_id=sessionid2
)

messages2 = run2.get_final_state()["messages"]
print("="*150)
print(get_final_answer(messages2))
print("="*150)
print(get_tools_called(messages2))
print("="*150)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
The first 3D platformer Mario game is **Super Mario 64**, released in 1996 for the Nintendo 64. This game was groundbreaking for its time and set new standards for the genre, featuring Mario's quest to rescue Princess Peach.
['retrieve_game', 'evaluate_retrieval']


In [11]:
query3="In which year was Black Myth Wukong was released for PS 5?"
sessionid3="wukong"

run3 = agent.invoke(
    query=query3,
    session_id=sessionid3
)

messages3 = run3.get_final_state()["messages"]
print("="*150)
print(get_final_answer(messages3))
print("="*150)
print(get_tools_called(messages3))
print("="*150)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
"Black Myth: Wukong" is set to be released for the PlayStation 5 on August 20, 2024. Additionally, a physical edition of the game will be available starting December 12, 2024. 

For more information, you can check the official PlayStation page [here](https://www.playstation.com/en-us/games/black-myth-wukong/).
['retrieve_game', 'evaluate_retrieval', 'game_web_search', 'evaluate_retrieval']


### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes